In [1]:
import pandas as pd
import sqlalchemy as db
import cx_Oracle as orcl

print("cx_Oracle.version:", orcl.version)


('cx_Oracle.version:', '7.2.3')


In [2]:
host = "10.21.18.13"
port = "1521"
sid = "ECHODB1"
dsn = orcl.makedsn(host, port, sid)
connection = orcl.connect("UAT","uat", dsn)
cur = orcl.Cursor(connection) 


In [ ]:
##checking customer information
statements = """SELECT * FROM CUSTOMER"""
customer = pd.read_sql_query(statements, connection)
pd.set_option('display.max_columns', None)
customer.head(4)

In [ ]:
###latest customer numbers
customer.tail(4)

In [ ]:
statement_trx = """select P.ACCOUNT_NUMBER,p.cust_id,
trim(C.SURNAME) || ' '|| trim(C.FIRST_NAME) ||' ' || trim(C.MIDDLE_NAME) as name,
D.ID_TRANSACT,
d.TIMESTAMP,
D.ID_JUSTIFIC,
J.DESCRIPTION,
DECODE (D.DEBIT_CREDIT_FLAG,'2','CR','1','DR') "DR/CR",
D.ENTRY_AMOUNT,
D.PREV_ACC_BALANCE, 
D.VALUE_DATE,
D.TRX_USR,
D.ENTRY_COMMENTS
from fst_demand_extrait d, justific j, profits_account p, customer c
where
D.ID_JUSTIFIC = J.ID_JUSTIFIC
and
P.CUST_ID = C.CUST_ID
and
D.FK_DEPOSIT_ACCOACC= P.DEP_ACC_NUMBER
and p.PRFT_SYSTEM=3
and trunc(d.TIMESTAMP) >= to_date('01-06-2019','dd-mm-yyyy')
and trunc(d.TIMESTAMP) <= to_date('30-09-2019','dd-mm-yyyy')
order by D.VALUE_DATE asc"""
trxns_data = pd.read_sql_query(statement_trx, connection)
pd.set_option('display.max_columns', None)
trxns_data.head(10)

In [7]:
#thiss has errors

statement_3 = """SELECT distinct D.TMSTAMP,
       D.TRX_DATE,
       d.TRX_UNIT,
       d.TRX_USR,
       d.TRX_USR_SN,
       d.TUN_INTERNAL_SN,
          TRIM (TO_CHAR (D.TRX_DATE, 'yyyymmdd'))
       || ''
       || TRIM (D.TRX_UNIT)
       || ''
       || TRIM (D.TRX_USR)
       || ''
       || TRIM (ABS (D.TRX_USR_SN) - 1)
       || ''
       || TRIM (D.TUN_INTERNAL_SN)
          AS TRANSACTION_REF,
       P.DESCRIPTION AS PRODUCT_DESCRIPTION,
       R.ACCOUNT_NUMBER,
       A.DESIGNATION AS ACCOUNT_NAME,
       D.I_ID_JUSTIFIC,
       J.DESCRIPTION AS TRANSACTION_TYPE,
       ABS (D.I_AMOUNT) "TRANSACTION_AMOUNT",
       D.ACC_AMOUNT_40 AS COMMISSION,
       D.I_COMMENTS TRX_COMMENTS,
       F.REVERSE_FLAG,
       F.REVERSED_TRX_FLAG
  FROM DEP_TRX_RECORDING D,
       JUSTIFIC J,
       PRODUCT P,
       DEPOSIT_ACCOUNT A,
       PROFITS_ACCOUNT R,
       FST_DEMAND_EXTRAIT F
WHERE    -- D.TRX_USR LIKE 'KOCE%'
       A.ACCOUNT_NUMBER = D.I_ACCOUNT_NUMBER
       AND R.DEP_ACC_NUMBER = D.I_ACCOUNT_NUMBER
       AND R.DEP_ACC_NUMBER = F.FK_DEPOSIT_ACCOACC
       AND A.ACCOUNT_NUMBER = F.FK_DEPOSIT_ACCOACC
       AND D.ID_PRODUCT = P.ID_PRODUCT
       AND D.I_ID_JUSTIFIC = J.ID_JUSTIFIC
       AND F.TRX_DATE = D.TRX_DATE
       AND F.TRX_UNIT = D.TRX_UNIT
       AND F.TRX_SN = D.TRX_USR_SN
       AND F.TRX_USR = D.TRX_USR
       AND F.TUN_INTERNAL_SN = D.TUN_INTERNAL_SN
       AND D.TRX_DATE BETWEEN TO_DATE ('01-01-2019', 'dd-mm-yyyy')
                          AND TO_DATE ('30-06-2019', 'dd-mm-yyyy')
       --AND D.I_ID_JUSTIFIC IN (31804,31805,31815,32096,32097,32099,32096,32097,32099)
       AND R.PRFT_SYSTEM = 3
       AND R.PRODUCT_ID IN (31102,
                            31103,
                            31104,
                            31112,
                            31113,
                            31501,
                            31511,
                            31521,
                            31531,
                            31541,
                            31551,
                            31561,
                            31572,
                            31703,
                            31704,
                            31705,
                            31718,
                            31719,
                            31721,
                            31730,
                            31731,
                            31732,
                            31750,
                            31761,
                            31771)
       AND F.REVERSE_FLAG = 0
       AND F.REVERSED_TRX_FLAG = 0
       AND D.ACC_AMOUNT_40 <> 0
       
       """
trxns_data_3 = pd.read_sql_query(statement_3, connection)
pd.set_option('display.max_columns', None)
trxns_data_3.head(10)


DatabaseError: Execution failed on sql: SELECT distinct D.TMSTAMP,
       D.TRX_DATE,
       d.TRX_UNIT,
       d.TRX_USR,
       d.TRX_USR_SN,
       d.TUN_INTERNAL_SN,
          TRIM (TO_CHAR (D.TRX_DATE, 'yyyymmdd'))
       || ''
       || TRIM (D.TRX_UNIT)
       || ''
       || TRIM (D.TRX_USR)
       || ''
       || TRIM (ABS (D.TRX_USR_SN) - 1)
       || ''
       || TRIM (D.TUN_INTERNAL_SN)
          AS TRANSACTION_REF,
       P.DESCRIPTION AS PRODUCT_DESCRIPTION,
       R.ACCOUNT_NUMBER,
       A.DESIGNATION AS ACCOUNT_NAME,
       D.I_ID_JUSTIFIC,
       J.DESCRIPTION AS TRANSACTION_TYPE,
       ABS (D.I_AMOUNT) "TRANSACTION_AMOUNT",
       D.ACC_AMOUNT_40 AS COMMISSION,
       D.I_COMMENTS TRX_COMMENTS,
       F.REVERSE_FLAG,
       F.REVERSED_TRX_FLAG
  FROM DEP_TRX_RECORDING D,
       JUSTIFIC J,
       PRODUCT P,
       DEPOSIT_ACCOUNT A,
       PROFITS_ACCOUNT R,
       FST_DEMAND_EXTRAIT F
WHERE    -- D.TRX_USR LIKE 'KOCE%'
       A.ACCOUNT_NUMBER = D.I_ACCOUNT_NUMBER
 

DatabaseError: Execution failed on sql 'SELECT D.TMSTAMP,
	      D.TRX_DATE,
	      --P.DESCRIPTION AS PRODUCT_DESCRIPTION, 
	      C.MOBILE_TEL,O.ID_NO, 
	      R.ACCOUNT_NUMBER,
              A.DESIGNATION AS ACCOUNT_NAME, 
              D.I_ID_JUSTIFIC,
             J.DESCRIPTION AS TRANSACTION_TYPE,
             D.O_FINAL_ACC_AMOUNT AS TRANSACTION_AMOUNT, 
	     D.ACC_AMOUNT_40 AS COMMISSION,D.I_COMMENTS , 
	    COUNT(D.O_FINAL_ACC_AMOUNT) AS COUNT_OF_TRANSACTIONS_MADE
	    SUM(D.O_FINAL_ACC_AMOUNT)	AS AMOUNTS_COLLECTED 
            SUM(D.ACC_AMOUNT_40) AS AMOUNT_EARNED_FROM_CUSTOMER
from DEP_TRX_RECORDING D, JUSTIFIC J, PRODUCT P, DEPOSIT_ACCOUNT A, PROFITS_ACCOUNT R, customer c,OTHER_ID O
WHERE 1=1
--D.TRX_USR = 'KOCE0001'
AND A.ACCOUNT_NUMBER = D.I_ACCOUNT_NUMBER
AND R.DEP_ACC_NUMBER = D.I_ACCOUNT_NUMBER
AND D.ID_PRODUCT = P.ID_PRODUCT
AND D.I_ID_JUSTIFIC = J.ID_JUSTIFIC
AND D.TRX_DATE between TO_DATE('01/01/2018', 'MM/DD/YYYY') and
TO_DATE('08/27/2019', 'MM/DD/YYYY')
--AND D.I_